In [31]:
import torch
import torchaudio
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchaudio.transforms import Convolve
from torchaudio.transforms import Resample
from torchaudio.functional import resample
from IPython.display import display, Audio
import numpy as np
import pandas as pd
import random
import os

In [16]:
aud_dir = '../../LibriVox_Kaggle/'
bg_dir = '../../LibriVox_Kaggle/BGnoise/'
rir_dir = '../../RIR/MIT_IR_Survey/Audio/'
train_csv_file = 'only_audioFname_train.csv'
test_csv_file = 'only_audioFname_test.csv'

bg_files = os.listdir(bg_dir)
rir_files = os.listdir(rir_dir)[1::]

SAMPLE_RATE = 16000

audResampler = Resample(22050, SAMPLE_RATE)
rirResampler = Resample(32000, SAMPLE_RATE)
bgResampler = Resample(44100, SAMPLE_RATE)

In [38]:
sample_aud = '../../LibriVox_Kaggle/achtgesichterambiwasse/achtgesichterambiwasse_0009.wav'
sample_rir = os.path.join(rir_dir,random.choice(rir_files))
sample_bg = os.path.join(bg_dir, random.choice(bg_files))

wav, sr = torchaudio.load(sample_aud)
rir, sr_rir = torchaudio.load(sample_rir)
bg, sr_bg = torchaudio.load(sample_bg)

wav00 = audResampler(wav)
rir00 = rirResampler(rir)
bg00 = torch.mean(bgResampler(bg), dim=0).unsqueeze(0)

wav01 = resample(wav, sr, SAMPLE_RATE)
rir01 = resample(rir, sr_rir, SAMPLE_RATE)
bg01 = torch.mean(resample(bg, sr_bg, SAMPLE_RATE), dim=0).unsqueeze(0)

In [39]:
convoluter = Convolve(mode='full')

convolved_audio = convoluter(wav00, rir00)[:,0:wav00.shape[1]]
noisy_aud = convolved_audio + bg00[:,0:convolved_audio.shape[1]]

convolved_audio0 = convoluter(wav01, rir01)[:,0:wav01.shape[1]]
noisy_aud0 = convolved_audio0 + bg01[:,0:convolved_audio0.shape[1]]

display(Audio(noisy_aud, rate=SAMPLE_RATE))
display(Audio(noisy_aud0, rate=SAMPLE_RATE))